In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import time, random, json, datetime, csv

# site url
naver_login_url = "https://nid.naver.com/nidlogin.login"
naver_url = 'https://www.naver.com/'

# file path
log_path = './log.csv'
config_path = './config.json'
comment_path = './comment.csv'

## read config json file 
try : 
    with open(config_path, 'r') as f: 
        config = json.load(f) 
except : 
    print('Check config.json path : ./config.json')

# User info
userid = config['USERINFO']["USERNAME"]
userpw = config['USERINFO']["PASSWORD"]
# now time
now_time = str(datetime.datetime.now())

# driver options
options = webdriver.ChromeOptions()
# options.add_argument('headless') # 백그라운드실행
# options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36")
# options.add_argument("lang=ko_KR") # 한국어!
# options.add_argument('window-size=1920x1080')
options.add_argument('no-sandbox')
options.add_argument('disable-dev-shm-usage')
options.add_argument("--disable-gpu")
driver = webdriver.Chrome('./chromedriver', chrome_options=options)


def naver_log_in(id, pw) :
    driver.get(naver_login_url)
    driver.implicitly_wait(10)
    time.sleep(random.uniform(1,1.5))
    # java script 
    driver.execute_script("document.getElementsByName('id')[0].value = \'" + id + "\'")
    time.sleep(random.uniform(1,1.5))
    driver.execute_script("document.getElementsByName('pw')[0].value=\'" + pw + "\'")
    time.sleep(random.uniform(1,1.5))
    driver.find_element_by_css_selector('.btn_global').click()
    time.sleep(3)

    naver_home = driver.current_url # 현재 주소 가져오기
    driver.implicitly_wait(10)
    
    if naver_home == naver_url :         
        print(now_time,'login_complete')
    else :
        print(now_time,'login_failed')
        driver.quit()   

# login 
try : 
    naver_log_in(userid, userpw) 
except : 
    print('Login fail : EXIT')


################## 꼬리 잡기 주소 입력 
comment_get_url = 'https://m.cafe.naver.com/ca-fe/web/cafes/30383037/articles/9?fromList=true'
##################


In [ ]:
# nuc_cafeid = '26163062'
# kan_cafeid = '29118241'
# my_cafeid = '30383037'

# # target_url = "https://m.cafe.naver.com/ca-fe/web/cafes/" + kan_cafeid 
# # target_url = "https://m.cafe.naver.com/ca-fe/web/cafes/" + nuc_cafeid 
# target_url = "https://m.cafe.naver.com/ca-fe/web/cafes/" + my_cafeid 

# # cafe 메인 페이지 이동 
# driver.get(target_url)
# driver.implicitly_wait(10)
# time.sleep(random.uniform(0.5,1))

In [ ]:
try : 
    for i in range(30) :
        driver.get(comment_get_url)
        time.sleep(0.5)
        body = driver.find_element_by_css_selector('body')
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.5)
        driver.find_element_by_xpath('//*[@id="ct"]/div[4]/div[2]/a/span[2]').click()

        # alert 창의 메시지를 확인하고 싶습니다.
        alert = driver.switch_to.alert
        message = alert.text
        print("Alert shows following message: "+ message )
        # time.sleep(random.uniform(0.5,0.8))

        # 메시지를 확인했으니 창을 닫습니다.
        alert = driver.switch_to.alert
        alert.accept()
        time.sleep(random.uniform(0.8,1))

except :
    print('Not alert' )
    pass

# 댓글 주소 이동
page_now = driver.current_url
page_split = page_now.split('?')[0]

driver.get(page_split)
driver.implicitly_wait(10)
# time.sleep(random.uniform(2,3))

def content_comment(xpath):
    elem = driver.find_element_by_xpath(xpath)
    elem.click()
    elem.send_keys('캠핑칸')
    
    elem.send_keys(Keys.RETURN)
    driver.find_element_by_css_selector('.CommentWriteUpload__btn_register').click()
    driver.implicitly_wait(10)
    time.sleep(random.uniform(2,3))

In [ ]:
# 댓글달기
for i in range(100) :
    start = time.time()  # 시작 시간 저장
    content_comment('//*[@id="cbox"]')
    print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

In [ ]:

driver.find_element_by_xpath('//*[@id="ct"]/div/div/ul/li['+str(i) +']/div/a[1]/strong').click()
driver.implicitly_wait(10)
time.sleep(random.uniform(2,3))
result = driver.current_url 
comment_board_numbers = result.split('articles/')[1] # 댓글 번호 분리
comment_board_numbers = comment_board_numbers.split('?fromList')[0] # 댓글 번호 분리
find_index_ = find_index(line, comment_board_numbers)
# print(comment_board_numbers)


if not find_index_ : 
    with open(log_path, 'a', encoding='utf-8', newline='') as fs :
        content_comment('//*[@id="cbox"]')
        
        wr = csv.writer(fs)
        wr.writerow([now_time, comment_board_numbers, userid, 'complete' ])
        print(now_time, comment_board_numbers, 'complete')
else :
    print(now_time, comment_board_numbers, 'already')
    time.sleep(random.uniform(2,3))
    driver.quit()
    break       

driver.quit()
print('Complete : EXIT')